In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

intTransformedLabels = encoder.fit_transform(labels)
print(intTransformedLabels)

In [ ]:
from numpy import array
from numpy import array
from numpy import argmax
from sklearn.preprocessing import OneHotEncoder

values = array(labels)
print(labels)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [ ]:
base_dir = '/media/ente/M2/2018 - 11 - sorted data'
labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax' ]
#class dirs
import os
ate_dir = os.path.join(base_dir, labels[0])
car_dir = os.path.join(base_dir, labels[1])
con_dir = os.path.join(base_dir, labels[2])
ede_dir = os.path.join(base_dir, labels[3])
eff_dir = os.path.join(base_dir, labels[4])
emp_dir = os.path.join(base_dir, labels[5])
fib_dir = os.path.join(base_dir, labels[6])
her_dir = os.path.join(base_dir, labels[7])
inf_dir = os.path.join(base_dir, labels[8])
mas_dir = os.path.join(base_dir, labels[9])
nof_dir = os.path.join(base_dir, labels[10])
nod_dir = os.path.join(base_dir, labels[11])
ple_dir = os.path.join(base_dir, labels[12])
pne_dir = os.path.join(base_dir, labels[13])
pn2_dir = os.path.join(base_dir, labels[14])
#filenames
ate_fnames = os.listdir(ate_dir)
car_fnames = os.listdir(car_dir)
con_fnames = os.listdir(con_dir)
ede_fnames = os.listdir(ede_dir)
eff_fnames = os.listdir(eff_dir)
emp_fnames = os.listdir(emp_dir)
fib_fnames = os.listdir(fib_dir)
her_fnames = os.listdir(her_dir)
inf_fnames = os.listdir(inf_dir)
mas_fnames = os.listdir(mas_dir)
nof_fnames = os.listdir(nof_dir)
nod_fnames = os.listdir(nod_dir)
ple_fnames = os.listdir(ple_dir)
pne_fnames = os.listdir(pne_dir)
pn2_fnames = os.listdir(pn2_dir)

print ('total Atelectasis images:       ', len(os.listdir(ate_dir))) 
print ('total Cardiomegaly images:      ', len(os.listdir(car_dir))) 
print ('total Consolidation images:     ', len(os.listdir(con_dir))) 
print ('total Edema images:             ', len(os.listdir(ede_dir))) 
print ('total Effusion images:          ', len(os.listdir(eff_dir))) 
print ('total Emphysema images:         ', len(os.listdir(emp_dir))) 
print ('total Fibrosis images:          ', len(os.listdir(fib_dir))) 
print ('total Hernia images:            ', len(os.listdir(her_dir))) 
print ('total Infiltration images:      ', len(os.listdir(inf_dir))) 
print ('total Mass images:              ', len(os.listdir(mas_dir))) 
print ('total No_Finding images:        ', len(os.listdir(nof_dir))) 
print ('total Nodule images:            ', len(os.listdir(nod_dir))) 
print ('total Pleural_Thickening images:', len(os.listdir(ple_dir))) 
print ('total Pneumonia images:         ', len(os.listdir(pne_dir))) 
print ('total Pneumothorax images:      ', len(os.listdir(pn2_dir))) 

## Split + Preprocessing + Augmentation

In [ ]:
batch_size = 32
rescale = 224
target_size = (rescale, rescale)
input_shape = (rescale, rescale, 3)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
    rescale=1./255,
    #rescale=((1./255)-.5)*2,
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=0.2) # set validation split

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb', 
    #classes=intTransformedLabels,
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    base_dir, # same directory as training data
    target_size=target_size,
    batch_size=batch_size,
    #early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto'),
    class_mode='categorical', #Determines the type of label arrays that are returned:"categorical" will be 2D one-hot encoded labels,
    color_mode='rgb',
    #classes=intTransformedLabels,
    subset='validation') # set as validation data

print(validation_generator.class_indices)
print(validation_generator.classes)

## Model Selection

In [3]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras import applications
from keras.optimizers import SGD
import keras_metrics as km

#layers + optimizer
#metrics= ['categorical_accuracy', km.precision(), km.recall()]
# build the VGG16 network
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (224,224,3))
model =  Sequential()
model.add(base_model)
model.add(Flatten())#model.add(Flatten(input_shape=base_model.output_shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
# and a logistic layer -- let's say we have 200 classes
model.add(Dense(15, activation = 'softmax'))

# set the first 25 layers (up to the last conv block) to non-trainable (weights will not be updated)
for layer in model.layers[:19]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=1e-4, momentum=0.9),metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 15)                15375     
Total params: 40,421,199
Trainable params: 0
Non-trainable params: 40,421,199
_________________________________________________________________


## Training Teil#1 (Pre-Training)

In [ ]:
history1=model.fit_generator(    
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    class_weight = 'balanced',
    epochs = epochs1)

## Bewertung des Pre-Trainings
### 1) Lernkurve betrachten

In [ ]:
from matplotlib import pyplot

pyplot.plot(history1.history['categorical_accuracy'])
pyplot.plot(history1.history['val_categorical_accuracy'])
pyplot.title('Training and validation accuracy')
pyplot.show()

pyplot.plot(history1.history['loss'])
pyplot.plot(history1.history['val_loss'])
pyplot.title('Training and validation loss')
pyplot.show()

### 2) Wahrheitsmatrix

In [ ]:
#Confution Matrix and Classification Report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred, target_names=labels))

## Export Model + Weights

Run the following cell to export results to disk.

In [ ]:
import h5py
export_dir= '/home/ente/Dropbox/THESIS/Code/Transfer_InceptionV3/Export'
model.save_weights(export_dir + 'my_model_weights#2.h5')

#### save as JSON
json_string = model.to_json()

#### save as YAML
yaml_string = model.to_yaml()

## Fine-Tuning
### Schichten auftauen

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the first convolution blocks, i.e. we will freeze
# the first 24 layers and unfreeze the rest:
for layer in model.layers[:15]:
   layer.trainable = False
for layer in model.layers[15:]:
   layer.trainable = True

## Training Teil#2 (Fine-Tuning)

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
#metrics= ['categorical_accuracy']
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Now let's retrain the model. We'll train on all images available, for 50 epochs, and validate on all test images. (This will take 2-3 hours to run.)

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history2=model.fit_generator(    
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    class_weight = 'balanced',
    epochs = epochs2)

## Bewertung des Fine-Tunings
### 1) Lernkurve

In [ ]:
from matplotlib import pyplot

pyplot.plot(history2.history['categorical_accuracy'])
pyplot.plot(history2.history['val_categorical_accuracy'])
pyplot.title('Training and validation accuracy')
pyplot.show()

pyplot.plot(history2.history['loss'])
pyplot.plot(history2.history['val_loss'])
pyplot.title('Training and validation loss')
pyplot.show()

### 2) Wahrheitsmatrix

In [ ]:
#Confution Matrix and Classification Report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred, target_names=labels))

### 3) Schichten visualisieren

In [ ]:
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img
# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(base_model.input, successive_outputs)

# Let's prepare a random input image of a cat or dog from the training set.
cat_img_files = [os.path.join(ate_dir, f) for f in ate_fnames]
dog_img_files = [os.path.join(car_dir, f) for f in car_fnames]
img_path = random.choice(cat_img_files + dog_img_files)

img = load_img(img_path, target_size=target_size)  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    pyplot.figure(figsize=(scale * n_features, scale))
    pyplot.title(layer_name)
    pyplot.grid(False)
    pyplot.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history1.history['categorical_accuracy']
val_acc = history1.history['val_categorical_accuracy']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history1.history['loss']
val_loss = history1.history['val_loss']

print(acc)
print(val_acc)
print(loss)
print(val_loss)

In [ ]:
# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history2.history['categorical_accuracy']
val_acc = history2.history['val_categorical_accuracy']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history2.history['loss']
val_loss = history2.history['val_loss']

print(acc)
print(val_acc)
print(loss)
print(val_loss)